# Import Libs & Data

In [1]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

2024-09-27 23:48:48.760904: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 23:48:48.765975: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 23:48:48.810431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 23:48:50.433168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')

In [3]:
print(concrete_data.columns.values)

['Cement' 'Blast Furnace Slag' 'Fly Ash' 'Water' 'Superplasticizer'
 'Coarse Aggregate' 'Fine Aggregate' 'Age' 'Strength']


In [4]:
# We can easily check the dataset for any missing values by using the following one-liner:  
concrete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


In [5]:
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# Data Analysis 

In [6]:
# Remove the results from the dataset (which is the target data)
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] 
# The target data is the result
target = concrete_data['Strength'] 

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head() # The target data "strength"

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
# Also we can normalize the data by substracting the mean and dividing by the standard deviation.

# Z = (X - μ) / σ, its a standard score of the sample (Gaussian distribution)
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
# Let's save the number of predictors to n_cols since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors

# Model Build

In [11]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=0)

# Display the shapes of the resulting datasets
print(f"Training predictors shape: {X_train.shape}")
print(f"Test predictors shape: {X_test.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Test target shape: {y_test.shape}")

Training predictors shape: (721, 8)
Test predictors shape: (309, 8)
Training target shape: (721,)
Test target shape: (309,)


In [12]:
# Keras model

# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [13]:
# build the model
model = regression_model()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# fit the model using only the training data
model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=1)

Epoch 1/100


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1619.5529 - val_loss: 1461.2347
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1583.8505 - val_loss: 1449.7642
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1600.8505 - val_loss: 1438.1793
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1656.4711 - val_loss: 1426.3862
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1554.3667 - val_loss: 1414.3331
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1553.1907 - val_loss: 1402.2122
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1515.5476 - val_loss: 1389.8475
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1483.0898 - val_loss: 1377.0168
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1524.8044 - val_loss: 1363.7095
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1467.8832 - val_loss: 1350.0618
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1520.4702 - val_loss: 1336.1875
Epoch 12/

# Predict the results

In [15]:
# Predict the results for the test set
y_pred = model.predict(X_test)

# Display the predicted results
print(y_pred)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[53.916878 ]
 [42.93249  ]
 [60.363987 ]
 [42.4932   ]
 [21.828661 ]
 [28.23236  ]
 [14.756437 ]
 [46.494587 ]
 [35.646587 ]
 [28.26639  ]
 [20.593943 ]
 [19.132952 ]
 [79.455185 ]
 [56.086433 ]
 [28.30991  ]
 [49.17028  ]
 [23.344713 ]
 [26.525858 ]
 [30.080364 ]
 [22.488546 ]
 [28.984632 ]
 [40.252186 ]
 [28.533619 ]
 [30.409698 ]
 [27.279467 ]
 [34.737835 ]
 [23.754604 ]
 [35.22936  ]
 [41.392063 ]
 [15.774068 ]
 [20.329252 ]
 [24.466217 ]
 [51.55389  ]
 [43.55922  ]
 [21.10281  ]
 [45.324913 ]
 [35.367775 ]
 [27.345106 ]
 [16.363798 ]
 [23.992195 ]
 [18.028393 ]
 [13.3544   ]
 [21.287378 ]
 [41.746952 ]
 [13.456695 ]
 [48.47065  ]
 [47.584362 ]
 [52.953922 ]
 [13.9976225]
 [20.484985 ]
 [36.55561  ]
 [27.748497 ]
 [30.754736 ]
 [33.184395 ]
 [32.4155   ]
 [42.649994 ]
 [34.57487  ]
 [21.56191  ]
 [32.41787  ]
 [22.727627 ]
 [26.924597 ]
 [20.014574 ]
 [22.760729 ]
 [49.34065  ]
 [17.787424 ]
 [24.520428 ]
 [24.571932 ]
 [21.72228  ]
 [18.157

In [16]:
# Compute the mean squared error between the predicted and actual concrete strength
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 184.87790922279422


# For the remaining 50-1 = 49 loops

In [17]:
# Initialize a list to store the mean squared errors
mse_list = [mse]

# Repeat the process 50 times
for i in range(1,50):
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    
    # Build the model
    model = regression_model()
    
    # Fit the model using only the training data
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    
    # Predict the results for the test set
    y_pred = model.predict(X_test)
    
    # Compute the mean squared error between the predicted and actual concrete strength
    mse = mean_squared_error(y_test, y_pred)
    
    # Append the mse to the list
    mse_list.append(mse)

# Calculate the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Mean of Mean Squared Errors: 624.9653072053329
Standard Deviation of Mean Squared Errors: 137.39971962777994
